In [0]:
pg_user = dbutils.secrets.get(scope="devDolphin", key="kushagra")
pg_pass = dbutils.secrets.get(scope="devDolphin", key="pg-password")

In [0]:
from pyspark.sql.functions import col, input_file_name
from datetime import datetime
import pyspark.sql.functions as F



hostname = "devdolphinpostgresdb.postgres.database.azure.com"
database = "postgres"

jdbc_url = f"jdbc:postgresql://{hostname}:5432/{database}?sslmode=require"

connection_properties = {
    "user": pg_user,
    "password": pg_pass,
    "driver": "org.postgresql.Driver"
}

# === ✅ Utility ===
def create_empty_tables():
    # Empty schema DFs
    gender_state_schema = "merchant STRING, customer STRING, gender STRING"
    pattern3_schema = """
      YStartTime TIMESTAMP,
      detectionTime TIMESTAMP,
      patternId STRING,
      ActionType STRING,
      customerName STRING,
      MerchantId STRING
    """

    spark.createDataFrame([], gender_state_schema) \
        .write.mode("overwrite").jdbc(jdbc_url, "gender_summary_state", properties=connection_properties)

    spark.createDataFrame([], pattern3_schema) \
        .write.mode("overwrite").jdbc(jdbc_url, "pattern3_detections", properties=connection_properties)

    print("✅ Tables created in Postgres.")

# Call once if needed:
# create_empty_tables()

# === ✅ Pattern 3 logic ===
def pattern3_detect(batch_df, batch_id):
    print(f"\n🔍 New batch {batch_id}")
    print(f"Rows: {batch_df.count()}")

    # Clean gender
    batch_df = batch_df.withColumn("gender", F.upper(F.trim(F.col("gender"))))

    # Keep required cols
    chunk_df = batch_df.select("merchant", "customer", "gender").dropna()

    # --- Load current state ---
    gender_state_df = spark.read.jdbc(jdbc_url, "gender_summary_state", properties=connection_properties)

    # Union new data & dedupe
    updated_state_df = (
        gender_state_df.unionByName(chunk_df)
        .dropDuplicates(["merchant", "customer", "gender"])
    )

    # === Gender conflict removal ===
    gender_counts = (
        updated_state_df.groupBy("merchant", "customer")
        .agg(F.countDistinct("gender").alias("gender_count"))
    )

    conflict_keys = (
        gender_counts.filter(col("gender_count") > 1)
        .select("merchant", "customer")
    )

    # Remove conflicts
    conflict_keys = conflict_keys.withColumn("join_key", F.concat_ws("_", "merchant", "customer"))
    updated_state_df = updated_state_df.withColumn("join_key", F.concat_ws("_", "merchant", "customer"))

    clean_state_df = updated_state_df.join(conflict_keys, on="join_key", how="left_anti").drop("join_key")

    # === Gender summary pivot ===
    gender_group = (
        clean_state_df.groupBy("merchant", "gender")
        .agg(F.countDistinct("customer").alias("customer_count"))
    )

    pivot_df = gender_group.groupBy("merchant") \
        .pivot("gender", ["F", "M"]) \
        .sum("customer_count") \
        .fillna(0)

    # Eligible pattern 3
    eligible_df = pivot_df.filter((col("F") > 100) & (col("F") < col("M")))

    if eligible_df.count() == 0:
        print("✅ No new Pattern 3 merchants.")
    else:
        print("🌟 Eligible DEI-NEEDED merchants:")
        eligible_df.show()

        now = datetime.now()

        detections_df = eligible_df.withColumn("YStartTime", F.lit(now)) \
            .withColumn("detectionTime", F.lit(now)) \
            .withColumn("patternId", F.lit("PatId3")) \
            .withColumn("ActionType", F.lit("DEI-NEEDED")) \
            .withColumn("customerName", F.lit("")) \
            .withColumnRenamed("merchant", "MerchantId") \
            .select(
                "YStartTime", "detectionTime", "patternId",
                "ActionType", "customerName", "MerchantId"
            )

        detections_df.write.mode("append").jdbc(jdbc_url, "pattern3_detections", properties=connection_properties)
        print(f"✅ Inserted {detections_df.count()} new detections.")

    # Write back cleaned state
    clean_state_df.write.mode("overwrite").jdbc(jdbc_url, "gender_summary_state", properties=connection_properties)
    print(f"✅ State table updated. Rows: {clean_state_df.count()}")


# === ✅ Start AutoLoader Stream ===
source_path = "abfss://gdrive-ingest@devdolphinstorage.dfs.core.windows.net/transactions"
schema_location = "abfss://gdrive-ingest@devdolphinstorage.dfs.core.windows.net/_schema/transactions"

stream_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.schemaLocation", schema_location)
    .load(source_path)
    .withColumn("source_file", input_file_name())
)

# === ✅ Hook up foreachBatch ===
query = (
    stream_df.writeStream
    .foreachBatch(pattern3_detect)
    .option("checkpointLocation", "abfss://gdrive-ingest@devdolphinstorage.dfs.core.windows.net/_checkpoint/pattern3")
    .start()
)

query.awaitTermination()


In [0]:
from pyspark.sql import SparkSession, functions as F
from datetime import datetime

#  Start Spark
spark = SparkSession.builder.getOrCreate()

#  Load CSV (adjust your path)
df = spark.read.csv("abfss://gdrive-ingest@devdolphinstorage.dfs.core.windows.net/transactions.csv", header=True, inferSchema=True)

print(f" Total rows loaded: {df.count()}")

#  Step 0: Clean gender (remove quotes, trim, upper)
df_clean = (
    df.withColumn(
        "gender",
        F.upper(F.trim(F.regexp_replace("gender", "'", "")))
    )
)

#  Step 1: Deduplicate (merchant, customer, gender)
deduped = (
    df_clean
    .select("merchant", "customer", "gender")
    .dropna()
    .dropDuplicates(["merchant", "customer", "gender"])
)

print(f"Deduped unique rows: {deduped.count()}")

# Step 2: Group by merchant + gender, count unique customers
gender_counts = (
    deduped
    .groupBy("merchant", "gender")
    .agg(F.countDistinct("customer").alias("customer_count"))
)

# Step 3: Pivot gender counts
pivot = (
    gender_counts
    .groupBy("merchant")
    .pivot("gender", ["F", "M"])
    .sum("customer_count")
    .fillna(0)
    .withColumnRenamed("F", "female_count")
    .withColumnRenamed("M", "male_count")
)

print("📊 Pivot sample:")
pivot.show(10, truncate=False)

# ✅ Step 4: Filter for pattern 3
filtered = pivot.filter(
    (F.col("female_count") > 100) & (F.col("female_count") < F.col("male_count"))
)

print("Eligible DEI-NEEDED merchants:")
filtered.show(truncate=False)

# Step 5: Construct detections DataFrame
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

detections = (
    filtered
    .withColumn("YStartTime", F.lit(now))
    .withColumn("detectionTime", F.lit(now))
    .withColumn("patternId", F.lit("PatId3"))
    .withColumn("ActionType", F.lit("DEI-NEEDED"))
    .withColumn("customerName", F.lit(""))
    .select("YStartTime", "detectionTime", "patternId", "ActionType", "customerName", "merchant")
    .withColumnRenamed("merchant", "MerchantId")
)

print(f"Total detections: {detections.count()}")
detections.show(truncate=False)

# Save or inspect as needed
detections.write.csv("abfss://gdrive-ingest@devdolphinstorage.dfs.core.windows.net/p3/", header=True, mode="overwrite")

print("🎉 Finished! ")
